In [1]:
!pip install ultralytics==8.0.134

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 4.9 MB/s eta 0:00:00


In [2]:
!wget https://github.com/raidtothesky/project-newton/raw/main/models/high-bias.pt

--2023-08-03 01:03:43--  https://github.com/raidtothesky/project-newton/raw/main/models/high-bias.pt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/raidtothesky/project-newton/main/models/high-bias.pt [following]
--2023-08-03 01:03:44--  https://raw.githubusercontent.com/raidtothesky/project-newton/main/models/high-bias.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6241262 (6.0M) [application/octet-stream]
Saving to: ‘high-bias.pt’

high-bias.pt        100%[===================>]   5.95M  --.-KB/s    in 0.1s    

2023-08-03 01:03:44 (54.7 MB/s) - ‘high-bias.pt’ saved [6241262/6241262]



In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

def crop_and_classify(model="high-bias.pt",
                      savepath="crop",
                      imagepath="https://storage.googleapis.com/648010c1-f244-4641-98f2-73ff6c1b4e99/ai_assignment_20230726_classify.jpeg",
                      plot=True):
  yolo_model = YOLO("high-bias.pt")
  results = yolo_model(imagepath)

  if os.path.exists(savepath):
    shutil.rmtree(savepath)

  results[0].save_crop(savepath)

  index = 1
  color_idx = {
      "red": 1,
      "green": 1,
      "yellow": 1,
  }

  for imgpath in [savepath + '/apple/' + path for path in os.listdir(savepath + '/apple') if path.endswith('.jpg')]:
      title, borimage = showcolorborders(imgpath)

      for c in color_idx:
        if not os.path.exists(savepath + '/apple/' + title + '/'):
          os.makedirs(savepath + '/apple/' + title + '/')

      cv2.imwrite(savepath + '/apple/' + title + '/' + title + '_' + str(color_idx[title]) + '.jpg', borimage)

      if plot:
        plt.figure(figsize = (20, 10))
        plt.subplot(5, 6, index)
        plt.title(title)
        plt.imshow(borimage[:,:,::-1])

      color_idx[title] = color_idx[title]+1
      index = index + 1
      # plt.show()

# the function below is largely repurposed from https://thinkinfi.com/color-detection-using-opencv-and-python/
# the key is to get a HSV range for each colors that are accurate enough for our test data
# because in this scope of work we don't have test data, it seems to work fine, but further test is necessary with more images.
#
# If I had the suitable data, I would fine-tune a classifier (e.g. based on VisionTransformer or CSPDarknet) to classify
# the colors of the apples. But I didn't find any. If I had more time, I would generate synthetic data
# of apples with three different colors with Stable Diffusion, then use them to train the classifier.
def showcolorborders(path):
    img = cv2.imread(path)
    input_image_cpy = img.copy()

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # HSV ranges for red green and yellow
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([7, 255, 255])

    lower_green = np.array([30, 20, 50])
    upper_green = np.array([90, 255, 255])

    lower_yellow = np.array([17, 20, 50])
    upper_yellow = np.array([29, 255, 255])

    # generating contour per color
    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    contours_red, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(mask_green, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # check which color has the largest continuous contour area
    max_area = 0
    max_color = "red"
    for cnt in contours_red:
        contour_area = cv2.contourArea(cnt)
        if contour_area > max_area:
            max_area = contour_area
            max_color = "red"

    for cnt in contours_green:
        contour_area = cv2.contourArea(cnt)
        if contour_area > max_area:
            max_area = contour_area
            max_color = "green"

    for cnt in contours_yellow:
        contour_area = cv2.contourArea(cnt)
        if contour_area > max_area:
            max_area = contour_area
            max_color = "yellow"
    return max_color, img

In [5]:
# you can specify the folder you want to save the classified images in. If not, a folder 'crop' will be created by default.
# if you are having problem with the PLT plotting, you can disable it by setting plot=False
crop_and_classify(plot=False)


Found https://storage.googleapis.com/648010c1-f244-4641-98f2-73ff6c1b4e99/ai_assignment_20230726_classify.jpeg locally at ai_assignment_20230726_classify.jpeg
image 1/1 /content/ai_assignment_20230726_classify.jpeg: 512x640 28 apples, 361.3ms
Speed: 5.4ms preprocess, 361.3ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
